In [1]:
from keras.utils import np_utils
from data import load_data

In [2]:
from __future__ import absolute_import
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from six.moves import range
from data import load_data
import random

Using Theano backend.
Using gpu device 0: GeForce GT 635M (CNMeM is disabled)
D:\Anaconda\lib\site-packages\theano-0.8.0.dev0-py2.7.egg\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [3]:
data, label = load_data()

In [4]:
from sklearn.cross_validation import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(data,label)

In [6]:
len(X_train)

15000

In [7]:
Y_train = np_utils.to_categorical(y_train,2)
Y_test = np_utils.to_categorical(y_test,2)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /=255

In [8]:
def make_network():
    model = Sequential()
    model.add(Convolution2D(32,3,3,border_mode='same',input_shape=(3,32,32)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32,3,3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64,3,3,border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model

In [9]:
batch_size = 32
nb_epoch = 200
def train_model(model,X_train,Y_train,X_test,Y_test):
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd)
    model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size,
             validation_split=0.1, show_accuracy=True, verbose=1)
    print('Testing...')
    res = model.evaluate(X_test, Y_test,
                        batch_size=batch_size, verbose=1, show_accuracy=True)
    print('Test accuracy: {0}'.format(res[1]))

In [10]:
model = make_network()

In [11]:
train_model(model,X_train,Y_train,X_test,Y_test)

Train on 13500 samples, validate on 1500 samples
Epoch 1/200
13500/13500 [==============================] - 104s - loss: 0.4326 - acc: 0.7988 - val_loss: 0.3267 - val_acc: 0.8667
Epoch 2/200
13500/13500 [==============================] - 105s - loss: 0.2413 - acc: 0.9054 - val_loss: 0.2032 - val_acc: 0.9140
Epoch 3/200
13500/13500 [==============================] - 106s - loss: 0.1706 - acc: 0.9330 - val_loss: 0.1172 - val_acc: 0.9547
Epoch 4/200
13500/13500 [==============================] - 110s - loss: 0.1223 - acc: 0.9541 - val_loss: 0.1026 - val_acc: 0.9600
Epoch 5/200
13500/13500 [==============================] - 111s - loss: 0.1128 - acc: 0.9576 - val_loss: 0.1227 - val_acc: 0.9527
Epoch 6/200
13500/13500 [==============================] - 105s - loss: 0.0980 - acc: 0.9627 - val_loss: 0.0769 - val_acc: 0.9740
Epoch 7/200
13500/13500 [==============================] - 105s - loss: 0.0830 - acc: 0.9691 - val_loss: 0.0733 - val_acc: 0.9733
Epoch 8/200
13500/13500 [================

In [14]:
model_json = model.to_json()
open('jianghuang_articture.json', 'w').write(model_json)
model.save_weights('jianhuang_weights.h5', overwrite=True)

In [16]:
import numpy as np
import scipy.misc
import os

In [23]:
def load_and_scale_imgs():
   img_names = os.listdir('imgs_p/')

   imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread('imgs_p/'+img_name), (32, 32)),
                        (2, 0, 1)).astype('float32')
           for img_name in img_names]
   return np.array(imgs) / 255

In [24]:
imgs = load_and_scale_imgs()

In [26]:
predictions = model.predict_classes(imgs)

10000/10000 [==============================] - 21s    


In [27]:
predictions

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [37]:
def predictImage(url):
    img_names = [url,]
    imgs = [np.transpose(scipy.misc.imresize(scipy.misc.imread(img_name), (32, 32)),
                        (2, 0, 1)).astype('float32')
           for img_name in img_names]
    imgs = np.array(imgs) / 255
    predictions = model.predict_classes(imgs)
    return predictions

In [39]:
predictImage('imgs_p/whgnhb1star003pl111.jpg')

1/1 [==============================] - 0s


array([1], dtype=int64)

In [40]:
predictImage('imgs_n/100065_01111.jpg')

1/1 [==============================] - 0s


array([0], dtype=int64)